# Mushroom

- 8124종류의 버섯의 특징과 독의 유무를 정리해논 데이터셋
- 버섯 모양의 특징을 기반으로 독의 유무를 판단

한줄(row)이 버섯 한 종류   
1번째 컬럼은 독의 유무 : 독(p), 식용(e)   
2번째 컬럼은 버섯의 머리모양 : b(벨형태), c(원뿔), x(블록), f(평평), k(혹), s(오목)   
4번째 컬럼은 버섯의 머리색 : n(갈색), b(황갈색), c(연한갈색), g(회색)   
자세한 내용은 사이트 참조

In [2]:
mr <- read.csv("../../R_class/Data/mushroom.csv", header = F, fileEncoding = "euc-kr")

In [15]:
# 인공신경망 , 로지스틱 회긔분석
library(nnet)
# 의사결정나무
library(party)
# 랜덤포레스트
library(randomForest)
# SVM
library(e1071)
# sampling
library(caret)

# arrange
library(dplyr)

Loading required package: grid
Loading required package: mvtnorm
Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: sandwich
randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: ‘ggplot2’

The following object is masked from ‘package:randomForest’:

    margin



In [4]:
str(mr)

'data.frame':	8124 obs. of  23 variables:
 $ V1 : Factor w/ 2 levels "e","p": 2 1 1 2 1 1 1 1 2 1 ...
 $ V2 : Factor w/ 6 levels "b","c","f","k",..: 6 6 1 6 6 6 1 1 6 1 ...
 $ V3 : Factor w/ 4 levels "f","g","s","y": 3 3 3 4 3 4 3 4 4 3 ...
 $ V4 : Factor w/ 10 levels "b","c","e","g",..: 5 10 9 9 4 10 9 9 9 10 ...
 $ V5 : Factor w/ 2 levels "f","t": 2 2 2 2 1 2 2 2 2 2 ...
 $ V6 : Factor w/ 9 levels "a","c","f","l",..: 7 1 4 7 6 1 1 4 7 1 ...
 $ V7 : Factor w/ 2 levels "a","f": 2 2 2 2 2 2 2 2 2 2 ...
 $ V8 : Factor w/ 2 levels "c","w": 1 1 1 1 2 1 1 1 1 1 ...
 $ V9 : Factor w/ 2 levels "b","n": 2 1 1 2 1 1 1 1 2 1 ...
 $ V10: Factor w/ 12 levels "b","e","g","h",..: 5 5 6 6 5 6 3 6 8 3 ...
 $ V11: Factor w/ 2 levels "e","t": 1 1 1 1 2 1 1 1 1 1 ...
 $ V12: Factor w/ 5 levels "?","b","c","e",..: 4 3 3 4 4 3 3 3 4 3 ...
 $ V13: Factor w/ 4 levels "f","k","s","y": 3 3 3 3 3 3 3 3 3 3 ...
 $ V14: Factor w/ 4 levels "f","k","s","y": 3 3 3 3 3 3 3 3 3 3 ...
 $ V15: Factor w/ 9 levels "b","c"

In [5]:
head(mr)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23
p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g


In [6]:
colSums(is.na(mr))

V1  V2  V3  V4  V5  V6  V7  V8  V9 V10 V11 V12 V13 V14 V15 V16 V17 V18 V19 V20 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
V21 V22 V23 
  0   0   0

In [9]:
# 분석하기 쉽게 문자를 숫자로 변환
mr[,2] <- as.numeric(mr[,2])
for(i in 2:ncol(mr)){
    mr[,i] <- as.numeric(mr[,i])
}

In [10]:
head(mr)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23
p,6,3,5,2,7,2,1,2,5,...,3,8,8,1,3,2,5,3,4,6
e,6,3,10,2,1,2,1,1,5,...,3,8,8,1,3,2,5,4,3,2
e,1,3,9,2,4,2,1,1,6,...,3,8,8,1,3,2,5,4,3,4
p,6,4,9,2,7,2,1,2,6,...,3,8,8,1,3,2,5,3,4,6
e,6,3,4,1,6,2,2,1,5,...,3,8,8,1,3,2,1,4,1,2
e,6,4,10,2,1,2,1,1,6,...,3,8,8,1,3,2,5,3,3,2


In [12]:
# V1 기준으로 정렬
mr <- arrange(mr,V1)
mr

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23
e,6,3,10,2,1,2,1,1,5,...,3,8,8,1,3,2,5,4,3,2
e,1,3,9,2,4,2,1,1,6,...,3,8,8,1,3,2,5,4,3,4
e,6,3,4,1,6,2,2,1,5,...,3,8,8,1,3,2,1,4,1,2
e,6,4,10,2,1,2,1,1,6,...,3,8,8,1,3,2,5,3,3,2
e,1,3,9,2,1,2,1,1,3,...,3,8,8,1,3,2,5,3,3,4
e,1,4,9,2,4,2,1,1,6,...,3,8,8,1,3,2,5,4,4,4
e,1,3,10,2,1,2,1,1,3,...,3,8,8,1,3,2,5,3,4,4
e,6,4,10,2,4,2,1,1,3,...,3,8,8,1,3,2,5,4,3,2
e,6,4,10,2,1,2,1,1,6,...,3,8,8,1,3,2,5,3,4,4
e,1,3,10,2,1,2,1,1,11,...,3,8,8,1,3,2,5,4,4,2


In [14]:
# 식용버섯의 갯수
nrow(subset(mr,V1 == "e"))
# 독버섯의 갯수
nrow(subset(mr,V1 == "p"))

[1] 4208

[1] 3916

In [16]:
#sampling
set.seed(1234)
samp <- createDataPartition(mr$V1, p = 0.7, list=F)

In [17]:
# 학습용 데이터
data.train <- mr[samp,]

In [18]:
# 검증용 데이터
data.test <- mr[-samp,]

In [19]:
# 문제와 정답 만들기
x <- subset(data.test, select = -V1)
y <- data.test$V1

In [20]:
# 학습시키기

# 인공신경망
model.nnet <- nnet(V1 ~ ., data=data.train, size = 3)

# 의사결정 나무
model.ctree <- ctree(V1 ~ ., data=data.train)

# 랜덤포레스트
model.randomForest <- randomForest(V1 ~ ., data=data.train, ntree=100)

#SVM
model.svm <- svm(V1 ~., data=data.train)

# 로지스틱 회귀분석
model.multinom <- multinom(V1 ~., data=data.train)

# weights:  73
initial  value 3751.869760 
iter  10 value 2164.495439
iter  20 value 1608.330000
iter  30 value 1462.655486
iter  40 value 1104.694997
iter  50 value 595.597184
iter  60 value 448.034489
iter  70 value 420.012301
iter  80 value 417.324883
iter  90 value 413.702682
iter 100 value 413.275018
final  value 413.275018 
stopped after 100 iterations


Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘V17’ constant. Cannot scale data.”

# weights:  24 (23 variable)
initial  value 3942.621163 
iter  10 value 1633.673376
iter  20 value 1264.137543
iter  30 value 868.476686
iter  40 value 789.424077
iter  50 value 785.517204
final  value 785.517135 
converged


In [21]:
# 검증하기

# 인공신경망
pred.nnet <- predict(model.nnet, x, type="class")

# 의사결정나무
pred.ctree <- predict(model.ctree, x)

# RandomForest
pred.randForest <- predict(model.randomForest, x)

#svm
pred.svm <- predict(model.svm, x)

# 로지스틱 회귀분석
pred.mutinom <- predict(model.multinom, x)

In [22]:
forecast <- rbind(mean(pred.nnet == y),
                  mean(pred.ctree == y),
                  mean(pred.randForest == y),
                  mean(pred.svm == y),
                  mean(pred.mutinom == y)
                 )
rownames(forecast) <- c("인공신경망", "의사결정나무", "랜덤포레스트", "SVM", "로지스틱 회귀분석")
colnames(forecast) <- c("예측력")
forecast

,예측력
인공신경망,0.9523810
의사결정나무,1.0000000
랜덤포레스트,1.0000000
SVM,1.0000000
로지스틱 회귀분석,0.9671593


In [25]:
# 오분류표
t(table(pred.nnet,y))
t(table(pred.mutinom,y))

   pred.nnet
y      e    p
  e 1262    0
  p  116 1058

   pred.mutinom
y      e    p
  e 1226   36
  p   44 1130